In [1]:
import sys
sys.path.append("../")
import pandas as pd
from utils.data_utils import DataLoader
from utils.encoding import CustomEncoder
import xgboost as xgb
import optuna
import lightgbm as lgb
from importlib import reload
from sklearn.metrics import roc_auc_score, accuracy_score
import json
import joblib
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestClassifier

reload(sys.modules['utils.data_utils'])
reload(sys.modules['utils.encoding'])

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/xgboost/compat.py:93: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


<module 'utils.encoding' from '/home/studio-lab-user/ML1-XAI/Project/notebooks/../utils/encoding.py'>

In [50]:
data_loader = DataLoader(file_path='../data/raw/car.data' , target_column='class', test_size=0.2, random_state=42)

In [51]:
X, y = data_loader.get_data()

X_ord, y = data_loader.get_data_ord()
X_ohe, y = data_loader.get_data_ohe()

In [52]:
X_train, y_train = data_loader.get_train_data()
X_test, y_test = data_loader.get_test_data()

X_train_ord, y_train_ord = data_loader.get_train_data_ord()
X_test_ord, y_test_ord = data_loader.get_test_data_ord()

X_train_ohe, y_train_ohe = data_loader.get_train_data_ohe()
X_test_ohe, y_test_ohe = data_loader.get_test_data_ohe()

In [5]:
print(X.shape, y.shape, '\n')

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape, '\n')

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape, '\n')

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape, '\n')

(1728, 6) (1728,) 

(1382, 6) (1382,)
(346, 6) (346,) 

(1382, 6) (1382,)
(346, 6) (346,) 

(1382, 6) (1382,)
(346, 6) (346,) 



# Hyperparameter Tuning

In [6]:
best_auc = {}

Following algorithms are used with both Ordinal & One-Hot Encoding

- XGBoost
- LGBM
- Random Forest
- CatBoost
- Neural Network (MLPClassifier)

## XGBoost - (Ordinal Encoded)

In [7]:
# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = X_train_ord, X_test_ord, y_train, y_test

def objective(trial):
    # Define the hyperparameters to tune
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    n_estimators = trial.suggest_int('n_estimators', 50, 1500)
    max_depth = trial.suggest_int('max_depth', 2, 20)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 10)
    subsample = trial.suggest_float('subsample', 0.1, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.1, 1.0)
    reg_alpha = trial.suggest_float('reg_alpha', 1e-7, 1e-2, log=True)
    reg_lambda = trial.suggest_float('reg_lambda', 1e-7, 1e-2, log=True)
    gamma = trial.suggest_float('gamma', 1e-7, 1e-2, log=True)
    max_delta_step = trial.suggest_int('max_delta_step', 0, 10)

    # Initialize the model with the hyperparameters
    model = xgb.XGBClassifier(learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth,
                              min_child_weight=min_child_weight, subsample=subsample, colsample_bytree=colsample_bytree,
                              reg_alpha=reg_alpha, reg_lambda=reg_lambda, gamma=gamma, max_delta_step=max_delta_step,
                              random_state=42)

    # Fit the model on the training set
    model.fit(X_train, y_train)

    # Predict the probabilities on the validation set
    y_proba = model.predict_proba(X_val)

    # Calculate the AUC-ROC score
    score = roc_auc_score(y_val, y_proba, multi_class='ovr')
    
    return score

# Set up the Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Print the best hyperparameters and score
print(f'Best hyperparameters: {study.best_params}')
print(f'Best score: {study.best_value}')

[I 2023-05-24 10:07:01,488] A new study created in memory with name: no-name-65624565-ec59-474f-ac83-1dee407958f2
[I 2023-05-24 10:07:02,979] Trial 0 finished with value: 0.9779289718410642 and parameters: {'learning_rate': 0.00013896483492993497, 'n_estimators': 1092, 'max_depth': 3, 'min_child_weight': 5, 'subsample': 0.21167858322904817, 'colsample_bytree': 0.9320767094964215, 'reg_alpha': 9.747046395877446e-06, 'reg_lambda': 0.006498235829760754, 'gamma': 3.8010039097665083e-06, 'max_delta_step': 1}. Best is trial 0 with value: 0.9779289718410642.
[I 2023-05-24 10:07:03,058] Trial 1 finished with value: 0.9546534017443169 and parameters: {'learning_rate': 0.003118678937547616, 'n_estimators': 66, 'max_depth': 16, 'min_child_weight': 2, 'subsample': 0.14370325429146225, 'colsample_bytree': 0.11876292428352914, 'reg_alpha': 1.1598547571063312e-05, 'reg_lambda': 2.9251396337660518e-05, 'gamma': 6.615159034360892e-06, 'max_delta_step': 6}. Best is trial 0 with value: 0.9779289718410642

Best hyperparameters: {'learning_rate': 0.06416766766469785, 'n_estimators': 1342, 'max_depth': 15, 'min_child_weight': 3, 'subsample': 0.8771588197922351, 'colsample_bytree': 0.9994140222437523, 'reg_alpha': 3.971072242931595e-06, 'reg_lambda': 3.740697452790934e-05, 'gamma': 0.0005851289483554137, 'max_delta_step': 10}
Best score: 1.0


Train model with complete data using best parameters

In [8]:
with open("../models/XGBoost/best_param_ord.json", "w") as json_file:
    json.dump(study.best_params, json_file)

model = xgb.XGBClassifier(**study.best_params)
model.fit(X_ord, y)

model_with_metadata = {
    'model': model,
    'metadata': {
        'description': 'XGBoost with input features encoded as Ordinal'
    }
}

joblib.dump(model_with_metadata, '../models/XGBoost/model_ord.sav')

['../models/XGBoost/model_ord.sav']

## XGBoost - (One Hot Encoded)

In [9]:
# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = X_train_ohe, X_test_ohe, y_train, y_test

def objective(trial):
    # Define the hyperparameters to tune
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    n_estimators = trial.suggest_int('n_estimators', 50, 1500)
    max_depth = trial.suggest_int('max_depth', 2, 20)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 10)
    subsample = trial.suggest_float('subsample', 0.1, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.1, 1.0)
    reg_alpha = trial.suggest_float('reg_alpha', 1e-7, 1e-2, log=True)
    reg_lambda = trial.suggest_float('reg_lambda', 1e-7, 1e-2, log=True)
    gamma = trial.suggest_float('gamma', 1e-7, 1e-2, log=True)
    max_delta_step = trial.suggest_int('max_delta_step', 0, 10)

    # Initialize the model with the hyperparameters
    model = xgb.XGBClassifier(learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth,
                              min_child_weight=min_child_weight, subsample=subsample, colsample_bytree=colsample_bytree,
                              reg_alpha=reg_alpha, reg_lambda=reg_lambda, gamma=gamma, max_delta_step=max_delta_step,
                              random_state=42)

    # Fit the model on the training set
    model.fit(X_train, y_train)

    # Predict the probabilities on the validation set
    y_proba = model.predict_proba(X_val)

    # Calculate the AUC-ROC score
    score = roc_auc_score(y_val, y_proba, multi_class='ovr')
    
    return score

# Set up the Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Print the best hyperparameters and score
print(f'Best hyperparameters: {study.best_params}')
print(f'Best score: {study.best_value}')

[I 2023-05-24 10:09:43,296] A new study created in memory with name: no-name-7f5beba7-1118-47fe-8ecd-f907e2b33851
[I 2023-05-24 10:09:46,404] Trial 0 finished with value: 0.986554103737882 and parameters: {'learning_rate': 0.00014273502873122574, 'n_estimators': 1409, 'max_depth': 12, 'min_child_weight': 2, 'subsample': 0.9399892695341502, 'colsample_bytree': 0.3276522817323859, 'reg_alpha': 0.0049373400443236855, 'reg_lambda': 0.0005796471975557656, 'gamma': 2.3190553924760217e-05, 'max_delta_step': 6}. Best is trial 0 with value: 0.986554103737882.
[I 2023-05-24 10:09:47,488] Trial 1 finished with value: 0.9862755650115761 and parameters: {'learning_rate': 0.00019616878016368215, 'n_estimators': 683, 'max_depth': 4, 'min_child_weight': 1, 'subsample': 0.8800415407074212, 'colsample_bytree': 0.42478835366858014, 'reg_alpha': 5.69073145653403e-06, 'reg_lambda': 0.00029472121748325044, 'gamma': 1.7141763467356224e-07, 'max_delta_step': 0}. Best is trial 0 with value: 0.986554103737882.


Best hyperparameters: {'learning_rate': 0.07581714982802396, 'n_estimators': 784, 'max_depth': 5, 'min_child_weight': 2, 'subsample': 0.9997742191948124, 'colsample_bytree': 0.732307395412825, 'reg_alpha': 0.00045581515635526927, 'reg_lambda': 3.3896013591768474e-06, 'gamma': 5.954075217412527e-07, 'max_delta_step': 10}
Best score: 0.9999879302853281


Train model with complete data using best parameters

In [10]:
with open("../models/XGBoost/best_param_ohe.json", "w") as json_file:
    json.dump(study.best_params, json_file)
    
model = xgb.XGBClassifier(**study.best_params)
model.fit(X_ohe, y)

model_with_metadata = {
    'model': model,
    'metadata': {
        'description': 'XGBoost with input features encoded as One Hot'
    }
}

joblib.dump(model_with_metadata, '../models/XGBoost/model_ohe.sav')

['../models/XGBoost/model_ohe.sav']

## LGBM (Ordinal Encoder)

In [11]:
# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = X_train_ord, X_test_ord, y_train, y_test

def objective(trial):
    # Define the hyperparameters to tune
    params = {
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'max_depth': trial.suggest_int('max_depth', 2, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 20),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-7, 1e-2, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-7, 1e-2, log=True),
        'min_split_gain': trial.suggest_float('min_split_gain', 1e-7, 1e-2, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1500)
    }

    # Initialize the model with the hyperparameters
    model = lgb.LGBMClassifier(**params, random_state=42)

    # Fit the model on the training set
    model.fit(X_train, y_train)

    # Predict the probabilities on the validation set
    y_proba = model.predict_proba(X_val)

    # Calculate the AUC-ROC score
    score = roc_auc_score(y_val, y_proba, multi_class='ovr')
    
    return score
# Set up the Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Print the best hyperparameters and score
print(f'Best hyperparameters: {study.best_params}')
print(f'Best score: {study.best_value}')

[I 2023-05-24 10:13:21,587] A new study created in memory with name: no-name-313c7ca2-2e9a-4844-a37a-5a9ad2f04624
[I 2023-05-24 10:13:23,068] Trial 0 finished with value: 0.9977430915413159 and parameters: {'learning_rate': 0.0009702690095088963, 'num_leaves': 255, 'max_depth': 6, 'min_child_samples': 4, 'subsample': 0.6724868179711272, 'colsample_bytree': 0.8144936649216392, 'reg_alpha': 6.744147710429969e-06, 'reg_lambda': 0.00032389458810379345, 'min_split_gain': 2.035026425975006e-07, 'n_estimators': 871}. Best is trial 0 with value: 0.9977430915413159.
[I 2023-05-24 10:13:24,256] Trial 1 finished with value: 0.9682235108923698 and parameters: {'learning_rate': 0.0009633856803300313, 'num_leaves': 47, 'max_depth': 20, 'min_child_samples': 7, 'subsample': 0.5074157321748798, 'colsample_bytree': 0.401411237550375, 'reg_alpha': 0.0013910079217128448, 'reg_lambda': 0.0026848323039728957, 'min_split_gain': 3.280498020020755e-05, 'n_estimators': 1030}. Best is trial 0 with value: 0.99774

Best hyperparameters: {'learning_rate': 0.01734691873018516, 'num_leaves': 65, 'max_depth': 12, 'min_child_samples': 1, 'subsample': 0.5315091991634423, 'colsample_bytree': 0.6709113755529368, 'reg_alpha': 0.002614857039907595, 'reg_lambda': 8.541695065718928e-05, 'min_split_gain': 0.0072252477973878074, 'n_estimators': 771}
Best score: 1.0


Train model with complete data using best parameters

In [12]:
with open("../models/LGBM/best_param_ord.json", "w") as json_file:
    json.dump(study.best_params, json_file)
    
model = lgb.LGBMClassifier(**study.best_params)
model.fit(X_ord, y)

model_with_metadata = {
    'model': model,
    'metadata': {
        'description': 'LGBM with input features encoded as Ordinal'
    }
}

joblib.dump(model_with_metadata, '../models/LGBM/model_ord.sav')

['../models/LGBM/model_ord.sav']

## LGBM (One Hot Encoding)

In [13]:
# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = X_train_ohe, X_test_ohe, y_train, y_test

def objective(trial):
    # Define the hyperparameters to tune
    params = {
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'max_depth': trial.suggest_int('max_depth', 2, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 20),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-7, 1e-2, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-7, 1e-2, log=True),
        'min_split_gain': trial.suggest_float('min_split_gain', 1e-7, 1e-2, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1500)
    }

    # Initialize the model with the hyperparameters
    model = lgb.LGBMClassifier(**params, random_state=42)

    # Fit the model on the training set
    model.fit(X_train, y_train)

    # Predict the probabilities on the validation set
    y_proba = model.predict_proba(X_val)

    # Calculate the AUC-ROC score
    score = roc_auc_score(y_val, y_proba, multi_class='ovr')
    
    return score
# Set up the Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Print the best hyperparameters and score
print(f'Best hyperparameters: {study.best_params}')
print(f'Best score: {study.best_value}')

[I 2023-05-24 10:14:52,063] A new study created in memory with name: no-name-5f776261-9ca7-49ae-b297-6478ae91f349
[I 2023-05-24 10:14:52,662] Trial 0 finished with value: 0.9674267903154428 and parameters: {'learning_rate': 0.0002537387331879056, 'num_leaves': 83, 'max_depth': 3, 'min_child_samples': 12, 'subsample': 0.6407557730217515, 'colsample_bytree': 0.6061888665092964, 'reg_alpha': 3.081949352797766e-06, 'reg_lambda': 2.9206898373026735e-05, 'min_split_gain': 0.0009071267246856372, 'n_estimators': 767}. Best is trial 0 with value: 0.9674267903154428.
[I 2023-05-24 10:14:55,531] Trial 1 finished with value: 0.9992278016409011 and parameters: {'learning_rate': 0.003642397447592525, 'num_leaves': 177, 'max_depth': 17, 'min_child_samples': 5, 'subsample': 0.25914707424378014, 'colsample_bytree': 0.43094709828894073, 'reg_alpha': 0.003450258138181448, 'reg_lambda': 0.0001817379392219894, 'min_split_gain': 2.6042105117396267e-07, 'n_estimators': 850}. Best is trial 1 with value: 0.999

Best hyperparameters: {'learning_rate': 0.025648312924308435, 'num_leaves': 10, 'max_depth': 5, 'min_child_samples': 6, 'subsample': 0.8032024523735697, 'colsample_bytree': 0.6556583992868715, 'reg_alpha': 6.378875667167661e-07, 'reg_lambda': 2.6420826567878003e-07, 'min_split_gain': 4.352868833706325e-07, 'n_estimators': 1108}
Best score: 1.0


Train model with complete data using best parameters

In [14]:
with open("../models/LGBM/best_param_ohe.json", "w") as json_file:
    json.dump(study.best_params, json_file)
    
model = lgb.LGBMClassifier(**study.best_params)
model.fit(X_ohe, y)

model_with_metadata = {
    'model': model,
    'metadata': {
        'description': 'LGBM with input features encoded as One Hot'
    }
}

joblib.dump(model_with_metadata, '../models/LGBM/model_ohe.sav')

['../models/LGBM/model_ohe.sav']

## Random Forest (Ordinal Encoder)

In [71]:
X_train, X_val, y_train, y_val = X_train_ord, X_test_ord, y_train, y_test

def objective(trial):
    # Define the hyperparameters to tune
    n_estimators = trial.suggest_int("n_estimators", 50, 1500)
    max_depth = trial.suggest_int("max_depth", 2, 20)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
    max_features = trial.suggest_categorical("max_features", ['sqrt', 'log2', None])
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    bootstrap = trial.suggest_categorical("bootstrap", [True, False])
    class_weight = trial.suggest_categorical("class_weight", [None, "balanced", "balanced_subsample"])

    # Initialize the model with the hyperparameters
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,
                                   min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,
                                   max_features=max_features, criterion=criterion, bootstrap=bootstrap,
                                   class_weight=class_weight, random_state=42)

    # Fit the model on the training set
    model.fit(X_train, y_train)

    # Predict the probabilities on the validation set
    y_proba = model.predict_proba(X_val)

    # Calculate the AUC-ROC score
    score = roc_auc_score(y_val, y_proba,  multi_class='ovr')

    # Return the negative AUC-ROC score, as Optuna minimizes the objective function
    return score

# Define the study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Print the best hyperparameters and score
print(f"Best hyperparameters: {study.best_params}")
print(f"Best auc_roc: {study.best_value}")

[I 2023-05-24 12:34:39,469] A new study created in memory with name: no-name-d5543569-e181-425f-8c3a-01b458b99ab1
[I 2023-05-24 12:34:42,747] Trial 0 finished with value: 0.999552495222485 and parameters: {'n_estimators': 1290, 'max_depth': 13, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'log2', 'criterion': 'entropy', 'bootstrap': False, 'class_weight': None}. Best is trial 0 with value: 0.999552495222485.
[I 2023-05-24 12:34:43,394] Trial 1 finished with value: 0.9998430937092647 and parameters: {'n_estimators': 222, 'max_depth': 20, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': None, 'criterion': 'gini', 'bootstrap': True, 'class_weight': 'balanced'}. Best is trial 1 with value: 0.9998430937092647.
[I 2023-05-24 12:34:44,168] Trial 2 finished with value: 0.9943287755764854 and parameters: {'n_estimators': 440, 'max_depth': 6, 'min_samples_split': 9, 'min_samples_leaf': 5, 'max_features': 'sqrt', 'criterion': 'gini', 'bootstrap': False, 'class_weig

Best hyperparameters: {'n_estimators': 179, 'max_depth': 16, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None, 'criterion': 'entropy', 'bootstrap': True, 'class_weight': None}
Best auc_roc: 0.9999577559986482


Train model with complete data using best parameters

In [72]:
with open("../models/Random_Forest/best_param_ord.json", "w") as json_file:
    json.dump(study.best_params, json_file)
    
model = RandomForestClassifier(**study.best_params)
model.fit(X_ord, y)

model_with_metadata = {
    'model': model,
    'metadata': {
        'description': 'RF with input features encoded as Ordinal'
    }
}

joblib.dump(model_with_metadata, '../models/Random_Forest/model_ord.sav')

['../models/Random_Forest/model_ord.sav']

## Random Forest (One Hot Encoding)

In [75]:
X_train, X_val, y_train, y_val = X_train_ohe, X_test_ohe, y_train, y_test

def objective(trial):
    # Define the hyperparameters to tune
    n_estimators = trial.suggest_int("n_estimators", 50, 1500)
    max_depth = trial.suggest_int("max_depth", 2, 20)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
    max_features = trial.suggest_categorical("max_features", ['sqrt', 'log2', None])
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    bootstrap = trial.suggest_categorical("bootstrap", [True, False])
    class_weight = trial.suggest_categorical("class_weight", [None, "balanced", "balanced_subsample"])

    # Initialize the model with the hyperparameters
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,
                                   min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,
                                   max_features=max_features, criterion=criterion, bootstrap=bootstrap,
                                   class_weight=class_weight, random_state=42)

    # Fit the model on the training set
    model.fit(X_train, y_train)

    # Predict the probabilities on the validation set
    y_proba = model.predict_proba(X_val)

    # Calculate the AUC-ROC score
    score = roc_auc_score(y_val, y_proba,  multi_class='ovr')

    # Return the negative AUC-ROC score, as Optuna minimizes the objective function
    return score

# Define the study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Print the best hyperparameters and score
print(f"Best hyperparameters: {study.best_params}")
print(f"Best auc_roc: {study.best_value}")

[I 2023-05-24 12:38:24,851] A new study created in memory with name: no-name-0cb5ff78-eeec-4be4-9e98-3b92a5577348
[I 2023-05-24 12:38:25,623] Trial 0 finished with value: 0.9988763912956162 and parameters: {'n_estimators': 163, 'max_depth': 18, 'min_samples_split': 8, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'criterion': 'entropy', 'bootstrap': False, 'class_weight': None}. Best is trial 0 with value: 0.9988763912956162.
[I 2023-05-24 12:38:27,380] Trial 1 finished with value: 0.9980086078588529 and parameters: {'n_estimators': 738, 'max_depth': 19, 'min_samples_split': 8, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'criterion': 'entropy', 'bootstrap': False, 'class_weight': 'balanced_subsample'}. Best is trial 0 with value: 0.9988763912956162.
[I 2023-05-24 12:38:28,984] Trial 2 finished with value: 0.9940032228959185 and parameters: {'n_estimators': 1028, 'max_depth': 8, 'min_samples_split': 6, 'min_samples_leaf': 10, 'max_features': 'log2', 'criterion': 'entropy', 'bootstra

Best hyperparameters: {'n_estimators': 583, 'max_depth': 13, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': 'log2', 'criterion': 'entropy', 'bootstrap': False, 'class_weight': 'balanced_subsample'}
Best auc_roc: 0.9994902972154578


Train model with complete data using best parameters

In [76]:
with open("../models/Random_Forest/best_param_ohe.json", "w") as json_file:
    json.dump(study.best_params, json_file)
    
model = RandomForestClassifier(**study.best_params)
model.fit(X_ohe, y)

model_with_metadata = {
    'model': model,
    'metadata': {
        'description': 'RF with input features encoded as One Hot'
    }
}

joblib.dump(model_with_metadata, '../models/Random_Forest/model_ohe.sav')

['../models/Random_Forest/model_ohe.sav']